# TSADGAN-SA for Tensorflow 2.0 

##Prepare1

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/salesforce/Merlion.git

Cloning into 'Merlion'...
remote: Enumerating objects: 2851, done.
remote: Counting objects: 100% (2711/2711), done.
remote: Compressing objects: 100% (1158/1158), done.
remote: Total 2851 (delta 2105), reused 2014 (delta 1553), pack-reused 140
Receiving objects: 100% (2851/2851), 87.19 MiB | 13.06 MiB/s, done.
Resolving deltas: 100% (2114/2114), done.
Checking out files: 100% (234/234), done.


In [ ]:
import sys
sys.path.append("/content/Merlion/ts_datasets")
sys.path.append("/content/Merlion/ts_datasets/ts_datasets")
sys.path.append('/content/drive/My Drive/LAC/Q2/implement/model/')

In [ ]:
import os
from anomaly.nab import NAB
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

from utils import plot, plot_ts, plot_rws, plot_error, unroll_ts

##Prepare2

In [ ]:
dataset_idx = 0
datasets = NAB(subset="realAWSCloudwatch")
time_series, metadata = datasets[dataset_idx]
X_train, y_train = time_series[metadata["trainval"]], metadata[metadata["trainval"]==True]["anomaly"]
X_test, y_test = time_series[~metadata["trainval"]], metadata[metadata["trainval"]==False]["anomaly"]
sum(y_train), sum(y_test)

Time series /content/Merlion/data/nab/realAWSCloudwatch/ec2_disk_write_bytes_1ef3de.csv (index 8) has timestamp duplicates. Kept first values.
Time series /content/Merlion/data/nab/realAWSCloudwatch/ec2_network_in_5abac7.csv (index 11) has timestamp duplicates. Kept first values.


(0, 402)

##Prepare3

In [ ]:
X_train, y_train = X_train[y_train!=True], y_train[y_train!=True]

In [ ]:
sum(y_train)

0

In [ ]:
#complement missing value
imp = SimpleImputer()
X_train = imp.fit_transform(X_train)
X_test = imp.fit_transform(X_test)

In [ ]:
len(X_train), len(y_train)

(605, 605)

In [ ]:
(np.nan in X_train), (np.nan in X_test)

(False, False)

In [ ]:
#Generator's output is between -1 and 1.
#So we need min-max scaling.
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(time_series)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  "X does not have valid feature names, but"


In [ ]:
max(X_train), min(X_train), max(X_test), min(X_test)

(array([0.28884987]), array([-1.]), array([1.]), array([-1.]))

In [ ]:
# def rolling_window_sequences(X, index, window_size, target_size, step_size, target_column,
#                              drop=None, drop_windows=False):
def rolling_window_sequences(X, y, window_size, step_size):
    """Create rolling window sequences out of time series data.
    The function creates an array of input sequences and an array of target sequences by rolling
    over the input sequence with a specified window.
    Optionally, certain values can be dropped from the sequences.
    Args:
        X (ndarray):
            N-dimensional sequence to iterate over.
        index (ndarray):
            Array containing the index values of X.
        window_size (int):
            Length of the input sequences.
        target_size (int):
            Length of the target sequences.
        step_size (int):
            Indicating the number of steps to move the window forward each round.
        target_column (int):
            Indicating which column of X is the target.
        drop (ndarray or None or str or float or bool):
            Optional. Array of boolean values indicating which values of X are invalid, or value
            indicating which value should be dropped. If not given, `None` is used.
        drop_windows (bool):
            Optional. Indicates whether the dropping functionality should be enabled. If not
            given, `False` is used.
    Returns:
        ndarray, ndarray, ndarray, ndarray:
            * input sequences.
            * target sequences.
            * first index value of each input sequence.
            * first index value of each target sequence.
    """
    out_X = list()
    out_y = list()
    # X_index = list()
    # y_index = list()
    # target = X[:, target_column]
    target = y.values

    # if drop_windows:
    #     if hasattr(drop, '__len__') and (not isinstance(drop, str)):
    #         if len(drop) != len(X):
    #             raise Exception('Arrays `drop` and `X` must be of the same length.')
    #     else:
    #         if isinstance(drop, float) and np.isnan(drop):
    #             drop = np.isnan(X)
    #         else:
    #             drop = X == drop

    start = 0
    max_start = len(X) - window_size + 1
    while start < max_start:
        end = start + window_size

        # if drop_windows:
        #     drop_window = drop[start:end + target_size]
        #     to_drop = np.where(drop_window)[0]
        #     if to_drop.size:
        #         start += to_drop[-1] + 1
        #         continue

        out_X.append(X[start:end])
        # out_y.append(target[end:end + target_size])
        if True in target[start:end]:
            out_y.append(True)
        else:
            out_y.append(False)
        # X_index.append(index[start])
        # y_index.append(index[end])
        start = start + step_size

    # return np.asarray(out_X), np.asarray(out_y), np.asarray(X_index), np.asarray(y_index)
    return np.asarray(out_X), np.asarray(out_y)

In [ ]:
X_train_ts, y_train_ts = rolling_window_sequences(X_train, y_train, window_size=100, step_size=1)

In [ ]:
X_test_ts, y_test_ts = rolling_window_sequences(X_test, y_test, window_size=100, step_size=1)

In [ ]:
anomaly_num = sum(y_test_ts)

##Prepare4

In [ ]:
print("Training data input shape: {}".format(X_train_ts.shape))
print("Training y shape: {}".format(y_train_ts.shape))

Training data input shape: (506, 100, 1)
Training y shape: (506,)


##Train all

###Train TadSAGAN

- GPU check for TadGAN 
- Load Tensorflow, Keras, Layers ..

In [ ]:
!nvidia-smi

Mon Feb  7 10:13:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from sklearn.metrics import classification_report
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

import tensorflow as tf
import keras
#import similaritymeasures as sm
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Bidirectional, LSTM, GRU, Flatten, Dense, Reshape, UpSampling1D, TimeDistributed, Conv1DTranspose
from tensorflow.keras.layers import Activation, Conv1D, LeakyReLU, Dropout, Add, Layer, Concatenate
from tensorflow.keras.optimizers import Adam

from functools import partial
from scipy import integrate, stats


In [ ]:
class SelfAttention(tf.keras.models.Model):
    '''
    Multi-head ではない単純な Self Attention
    '''

    def __init__(self, depth: int, *args, **kwargs):
        '''
        コンストラクタ。
        :param depth: 隠れ層及び出力の次元
        '''
        super().__init__(*args, **kwargs)
        self.depth = depth

        self.q_dense_layer = tf.keras.layers.Dense(depth, use_bias=False, name='q_dense_layer')
        self.k_dense_layer = tf.keras.layers.Dense(depth, use_bias=False, name='k_dense_layer')
        self.v_dense_layer = tf.keras.layers.Dense(depth, use_bias=False, name='v_dense_layer')
        self.output_dense_layer = tf.keras.layers.Dense(depth, use_bias=False, name='output_dense_layer')

    def call(self, input: tf.Tensor) -> tf.Tensor:
        '''
        モデルの実行。
        :param input: query のテンソル
        :param memory: query に情報を与える memory のテンソル
        '''
        q = self.q_dense_layer(input)  # [batch_size, q_length, depth]
        k = self.k_dense_layer(input)  # [batch_size, m_length, depth]
        v = self.v_dense_layer(input)

        # ここで q と k の内積を取ることで、query と key の関連度のようなものを計算。
        logit = tf.matmul(q, k, transpose_b=True)  # [batch_size, q_length, k_length]

        # softmax を取ることで正規化
        attention_weight = tf.nn.softmax(logit, name='attention_weight')

        # 重みに従って value から情報を引く
        attention_output = tf.matmul(attention_weight, v)  # [batch_size, q_length, depth]
        return self.output_dense_layer(attention_output)

In [ ]:
#データ取ったのは、GとEのLeakyReLUは存在しないバージョン
def build_encoder_layer(input_shape, encoder_reshape_shape):
    x = Input(shape=input_shape)
    model = tf.keras.models.Sequential([
        Conv1D(64,5), #元は64,5
        # LSTM(units=100, return_sequences=True), 
        SelfAttention(depth=64),
        Dense(64, activation=LeakyReLU(alpha=0.2)),
        Flatten(),
        Dense(20, activation=LeakyReLU(alpha=0.2)),
        Reshape(target_shape=encoder_reshape_shape)])
    return Model(x, model(x))

def build_generator_layer(input_shape, generator_reshape_shape):
    x = Input(shape=input_shape)
    model = tf.keras.models.Sequential([
        Flatten(),
        Dense(50, activation=LeakyReLU(alpha=0.2)), #50
        Reshape(target_shape=(50,1)),  # (50, 1)
        LSTM(units=64, return_sequences=True),
        Dropout(rate=0.2),
        UpSampling1D(size=2),
        LSTM(units=64, return_sequences=True),
        Dropout(rate=0.2),
        SelfAttention(depth=64),
        Dense(32, activation=LeakyReLU(alpha=0.2)),
        TimeDistributed(Dense(1, activation=LeakyReLU(alpha=0.2))),
        Activation(activation='tanh')
        ])  # (None, 100, 1)

    return Model(x, model(x))

def build_critic_layer(x_input_shape, z_input_shape):

    #C(x)
    x_in = Input(shape=x_input_shape)
    x = Conv1D(filters=32, kernel_size=5)(x_in) #64
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(rate=0.5)(x)
    x = Flatten()(x)
    
    #C(z)
    z_in = Input(shape=z_input_shape)
    z = Flatten()(z_in)
    z = Dense(units=32)(z) #100
    z = LeakyReLU(alpha=0.2)(z)
    z = Dropout(rate=0.2)(z)

    x_z = Concatenate(axis=1)([x, z])

    x_z = Dense(units=32)(x_z) #128
    fd = LeakyReLU(alpha=0.2)(x_z)
    x_z = Dense(units=1)(fd)
    output = Activation("sigmoid")(x_z)

    return Model([x_in, z_in],[fd, output])

In [ ]:
# Layer Parameters
latent_dim = 20 #20
shape = (100, 1)

encoder_input_shape = (100, 1)
generator_input_shape = (20, 1) #
critic_x_input_shape = (100, 1)
critic_z_input_shape = (20,1) #
encoder_reshape_shape = (20, 1) #
generator_reshape_shape = (50, 1)
learning_rate = 0.0005

encoder = build_encoder_layer(input_shape=encoder_input_shape, encoder_reshape_shape=encoder_reshape_shape)
generator = build_generator_layer(input_shape=generator_input_shape, generator_reshape_shape=generator_reshape_shape)
critic = build_critic_layer(x_input_shape=critic_x_input_shape, z_input_shape=critic_z_input_shape)

optimizer = tf.keras.optimizers.Adam(learning_rate)

In [ ]:
# tf.compat.v1.disable_eager_execution()

In [ ]:
z = Input(shape=(latent_dim, 1))
x = Input(shape=shape)
x_ = generator(z)
z_ = encoder(x)
fake_fd, fake_x_z = critic([x_, z]) #Generator input & outputを入力 <- criticの学習ではこっちに騙されないように
valid_fd, valid_x_z = critic([x, z_]) #Encoder input & outputを入力 <- こっちを本物だと思うように学習

critic_model = Model(inputs=[x, z], outputs=[fake_x_z, valid_x_z, fake_fd, valid_fd])

In [ ]:
z_gen = Input(shape=(latent_dim, 1))
x_gen_ = generator(z_gen)
x_enc = Input(shape=shape)
z_enc_ = encoder(x_enc)
x_gen_rec = generator(z_enc_)
fake_enc_gen_fd, fake_enc_gen_x_z = critic([x_gen_, z_gen]) #Generator input & outputを入力 <- Generatorの学習ではこっちを本物のように
valid_enc_gen_fd, valid_enc_gen_x_z = critic([x_enc, z_enc_]) #Encoder input & outputを入力 <- Encoderの学習ではこっちを偽物のように学習

encoder_generator_model = Model([x_enc, z_gen], [fake_enc_gen_x_z, valid_enc_gen_x_z, x_gen_rec, fake_enc_gen_fd, valid_enc_gen_fd])

In [ ]:
@tf.function
def critic_train_on_batch(x, z, valid, fake):
    with tf.GradientTape() as tape:
        
        fake_x_z, valid_x_z, _, _ = critic_model(inputs=[x, z], training=True) 

        # if np.random.random(1) < 0.8:
        #     loss = tf.reduce_mean(tf.keras.metrics.binary_crossentropy(valid, valid_x_z))
        #     loss += tf.reduce_mean(tf.keras.metrics.binary_crossentropy(fake, fake_x_z))
        # else:
        #     loss = tf.reduce_mean(tf.keras.metrics.binary_crossentropy(fake, valid_x_z))
        #     loss += tf.reduce_mean(tf.keras.metrics.binary_crossentropy(valid, fake_x_z))
        loss = tf.reduce_mean(tf.keras.metrics.binary_crossentropy(valid, valid_x_z))
        loss += tf.reduce_mean(tf.keras.metrics.binary_crossentropy(fake, fake_x_z))
        
    gradients = tape.gradient(loss, critic_model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, critic_model.trainable_weights))
    return loss

In [ ]:
@tf.function
def enc_gen_train_on_batch(x, z, valid):
    with tf.GradientTape() as tape:
        
        fake_enc_gen_x_z, valid_enc_gen_x_z, x_gen_rec, _, _ = encoder_generator_model(inputs=[x, z], training=True)
        
        x = tf.squeeze(x)
        x_gen_rec = tf.squeeze(x_gen_rec)
        
        loss = tf.reduce_mean(tf.keras.metrics.binary_crossentropy(valid, fake_enc_gen_x_z))
        loss += tf.reduce_mean(tf.keras.metrics.binary_crossentropy(fake, valid_enc_gen_x_z))
        loss += tf.keras.losses.MSE(x, x_gen_rec)*10 #10
        loss = tf.reduce_mean(loss)
        
    gradients = tape.gradient(loss, encoder_generator_model.trainable_weights)
    optimizer.apply_gradients(zip(gradients, encoder_generator_model.trainable_weights))
    
    return loss

In [ ]:
# Train parameters
batch_size = 64
n_critics = 1
epochs = 50

In [ ]:
c_losses = []
g_losses = []

In [ ]:
# Train 
X_train_ts_ = np.copy(X_train_ts)

fake = np.zeros((batch_size, 1), dtype=np.float32)
valid = np.ones((batch_size, 1), dtype=np.float32)

tsg_best_cr = 0
tsg_best_f1 = 0.0

for epoch in range(1, epochs+1):
    
    np.random.shuffle(X_train_ts_)
    
    epoch_g_loss = []
    epoch_c_loss = []
    
    minibatches_size = batch_size * n_critics
    num_minibatches = int(X_train_ts_.shape[0] // minibatches_size)
    
    for i in range(num_minibatches):
        minibatch = X_train_ts_[i * minibatches_size: (i + 1) * minibatches_size]
        
        for j in range(n_critics):
            #train critic
            generator.trainable = False
            encoder.trainable = False
            critic.trainable = True 
            x = minibatch[j * batch_size: (j + 1) * batch_size]
            z = np.random.normal(size=(batch_size, latent_dim, 1))
            epoch_c_loss.append(critic_train_on_batch(x, z, valid, fake))
            
            # train encoder, generator  
            critic.trainable = False
            generator.trainable = True
            encoder.trainable = True   
            for _ in range(10): # 10 → 5
                epoch_g_loss.append(enc_gen_train_on_batch(x, z, valid))

    plt.plot(X_train_ts[0])
    plt.plot(generator(encoder(X_train_ts[0].reshape(1,100,1)))[0])
    plt.show() 
        
    c_loss = np.mean(np.array(epoch_c_loss), axis=0)
    g_loss = np.mean(np.array(epoch_g_loss), axis=0)
    print('Epoch: {}/{}, [C loss: {}] [G loss: {}]'.format(epoch, epochs, c_loss, g_loss))

    if epoch%1 == 0:
        # Get test MSE loss.
        test_enc_out = encoder.predict(X_test_ts)
        X_test_ts_pred = generator(test_enc_out)

        #reconstruct loss
        mse_losses = np.mean(np.mean(np.square(X_test_ts_pred - X_test_ts), axis=-1),axis=-1)
        mae_losses = np.mean(np.mean(np.abs(X_test_ts_pred - X_test_ts), axis=-1),axis=-1)
        # dtw_losses = []
        # for pred_ts, test_ts in zip(X_test_ts_pred, X_test_ts):
        #     dtw_loss, _ = fastdtw(pred_ts, test_ts, dist=euclidean) #dtw
        #     dtw_losses.append(dtw_loss)
        mse_losses = stats.zscore(mse_losses)
        mae_losses = stats.zscore(mae_losses)
        # dtw_losses = stats.zscore(dtw_losses)
        mse_losses = np.clip(mse_losses, a_min=0, a_max=None) + 1
        mae_losses = np.clip(mae_losses, a_min=0, a_max=None) + 1
        # dtw_losses = np.clip(dtw_losses, a_min=0, a_max=None) + 1

        #critic loss
        test_fake_fd_val, _ = critic([X_test_ts_pred, test_enc_out])
        test_valid_fd_val, _ = critic([X_test_ts, test_enc_out])
        test_critic_scores = np.mean(np.square(test_fake_fd_val - test_valid_fd_val), axis=-1)
        test_critic_scores = stats.zscore(test_critic_scores)
        test_critic_scores = np.clip(test_critic_scores, a_min=0, a_max=None) + 1

        #combination loss
        # test_anomaly_scores = test_critic_loss * np.array(rec_loss)
        # test_anomaly_scores = test_critic_loss + np.array(rec_loss)

        #anomaly detection
        ##rec_loss
        ###mse
        test_pred = np.zeros(len(y_test_ts))
        anomalies_idx = np.argsort(mse_losses)[::-1][0:anomaly_num]
        test_pred[anomalies_idx] = 1
        # print("mse")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        if f1_score(y_test_ts, test_pred) > tsg_best_f1:
            tsg_best_f1 = f1_score(y_test_ts, test_pred)
            tsg_best_cr = classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4)
        ###mae
        test_pred = np.zeros(len(y_test_ts))
        anomalies_idx = np.argsort(mae_losses)[::-1][0:anomaly_num]
        test_pred[anomalies_idx] = 1
        # print("mae")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        if f1_score(y_test_ts, test_pred) > tsg_best_f1:
            tsg_best_f1 = f1_score(y_test_ts, test_pred)
            tsg_best_cr = classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4)
        ###dtw
        # test_pred = np.zeros(len(y_test_ts))
        # anomalies_idx = np.argsort(dtw_losses)[::-1][0:anomaly_num]
        # test_pred[anomalies_idx] = 1
        # print("dtw")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        ##critic_loss
        test_pred = np.zeros(len(y_test_ts))
        anomalies_idx = np.argsort(test_critic_scores)[::-1][0:anomaly_num]
        test_pred[anomalies_idx] = 1
        # print("critic")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        if f1_score(y_test_ts, test_pred) > tsg_best_f1:
            tsg_best_f1 = f1_score(y_test_ts, test_pred)
            tsg_best_cr = classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4)
        ##combination_loss
        ###mse
        test_pred = np.zeros(len(y_test_ts))
        test_anomaly_scores = test_critic_scores * np.array(mse_losses)
        anomalies_idx = np.argsort(test_anomaly_scores)[::-1][0:anomaly_num]
        test_pred[anomalies_idx] = 1
        # print("mse x critic")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        if f1_score(y_test_ts, test_pred) > tsg_best_f1:
            tsg_best_f1 = f1_score(y_test_ts, test_pred)
            tsg_best_cr = classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4)
        test_pred = np.zeros(len(y_test_ts))
        test_anomaly_scores = test_critic_scores + np.array(mse_losses)
        anomalies_idx = np.argsort(test_anomaly_scores)[::-1][0:anomaly_num]
        test_pred[anomalies_idx] = 1
        # print("mse + critic")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        if f1_score(y_test_ts, test_pred) > tsg_best_f1:
            tsg_best_f1 = f1_score(y_test_ts, test_pred)
            tsg_best_cr = classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4)
        ###mae
        test_pred = np.zeros(len(y_test_ts))
        test_anomaly_scores = test_critic_scores * np.array(mae_losses)
        anomalies_idx = np.argsort(test_anomaly_scores)[::-1][0:anomaly_num]
        test_pred[anomalies_idx] = 1
        # print("mae x critic")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        if f1_score(y_test_ts, test_pred) > tsg_best_f1:
            tsg_best_f1 = f1_score(y_test_ts, test_pred)
            tsg_best_cr = classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4)
        test_pred = np.zeros(len(y_test_ts))
        test_anomaly_scores = test_critic_scores + np.array(mae_losses)
        anomalies_idx = np.argsort(test_anomaly_scores)[::-1][0:anomaly_num]
        test_pred[anomalies_idx] = 1
        # print("mae + critic")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        if f1_score(y_test_ts, test_pred) > tsg_best_f1:
            tsg_best_f1 = f1_score(y_test_ts, test_pred)
            tsg_best_cr = classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4)
        ###dtw
        # test_pred = np.zeros(len(y_test_ts))
        # test_anomaly_scores = test_critic_scores * np.array(dtw_losses)
        # anomalies_idx = np.argsort(test_anomaly_scores)[::-1][0:anomaly_num]
        # test_pred[anomalies_idx] = 1
        # print("dtw x critic")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
        # test_pred = np.zeros(len(y_test_ts))
        # test_anomaly_scores = test_critic_scores + np.array(dtw_losses)
        # anomalies_idx = np.argsort(test_anomaly_scores)[::-1][0:anomaly_num]
        # test_pred[anomalies_idx] = 1
        # print("dtw + critic")
        # print(classification_report(y_test_ts, test_pred, target_names=["正常","異常"], digits=4))
 
    c_losses.append(c_loss)
    g_losses.append(g_loss)
print(tsg_best_f1)
print(tsg_best_cr)

Output hidden; open in https://colab.research.google.com to view.

##Result

In [ ]:
print("tsg")
print(tsg_best_cr)

ae
              precision    recall  f1-score   support

          正常     0.9417    0.9417    0.9417     12555
          異常     0.1202    0.1202    0.1202       832

    accuracy                         0.8906     13387
   macro avg     0.5309    0.5309    0.5309     13387
weighted avg     0.8906    0.8906    0.8906     13387

tg
              precision    recall  f1-score   support

          正常     0.9451    0.9451    0.9451     12555
          異常     0.1719    0.1719    0.1719       832

    accuracy                         0.8971     13387
   macro avg     0.5585    0.5585    0.5585     13387
weighted avg     0.8971    0.8971    0.8971     13387

tsg
              precision    recall  f1-score   support

          正常     0.9505    0.9505    0.9505     12555
          異常     0.2524    0.2524    0.2524       832

    accuracy                         0.9071     13387
   macro avg     0.6014    0.6014    0.6014     13387
weighted avg     0.9071    0.9071    0.9071     13387

